# Phase 3 - Normal Vector Stability Analysis

## Overview

This notebook analyzes the stability of normal vectors computed from a reference plane defined by 3 landmark nodes. It quantifies stability metrics, measures directional changes over time, and provides longer-period state estimates. Additionally, it performs across-frame distribution analysis, worst-case identification, and nose point stability analysis.

## What This Analysis Does

The analysis computes plane normal vectors from 3 landmark points (typically head landmarks) and measures:
1. **Normal Vector Stability**: How stable the plane orientation is over time
2. **Angle Changes**: Frame-to-frame angular changes in the normal vector direction
3. **Angular Velocity**: Rate of change of normal vector direction
4. **Rolling Statistics**: Mean normal vectors over different time windows (0.5s, 1s, 5s)
5. **State Classification**: Classifies each frame as "stable" or "changing" based on stability metrics
6. **Per-Second Analysis**: Mean normal vector per second and angle shifts between seconds
7. **Across-Frame Distribution**: Statistical distributions of all metrics with skewness and kurtosis
8. **Worst-Case Analysis**: Identifies outlier frames, maximum deviations, and unstable periods
9. **Nose Point Stability**: Dedicated analysis of nose point (node_3) positional stability

## Methodology

### 1. Plane Normal Computation
- For each frame, compute the plane normal vector from 3 landmark points using cross product
- Normalize the vector to unit length
- The normal vector represents the orientation of the reference plane

### 2. Angle Change Metrics
- **Frame-to-frame angle change**: Angle between consecutive normal vectors
- **Cumulative angle change**: Total angle change from the first valid frame
- **Angular velocity**: Rate of change in degrees per second

### 3. Rolling Statistics
- Compute rolling mean normal vectors over windows of 0.5s, 1s, and 5s
- Compute rolling standard deviation of angles from the rolling mean
- These provide smoothed estimates of normal vector direction

### 4. Low-Pass Filtering
- Apply Butterworth low-pass filter (5 Hz cutoff) to reduce high-frequency noise
- Helps separate real directional changes from tracking noise

### 5. State Classification
- Classify frames as "stable" or "changing" based on rolling standard deviation
- Threshold: < 5 degrees = stable, >= 5 degrees = changing

### 6. Per-Second Block Analysis
- Compute mean normal vector for each 1-second block
- Compute angle shift between consecutive seconds
- Enforces continuity to avoid 180-degree jumps

### 7. Across-Frame Distribution Analysis
- Compute full statistical distributions for angle changes, angular velocity, and normal vector components
- Calculate percentiles (25th, 75th, 95th, 99th), skewness, and kurtosis
- Helps characterize the shape and spread of metric distributions

### 8. Worst-Case Analysis
- **Maximum deviation**: Largest angle deviation from mean normal vector
- **Outlier detection**: Frames with angle changes exceeding 3 standard deviations
- **Unstable periods**: Contiguous time periods where stability std exceeds threshold
- **Maximum angular velocity**: Frames with highest rate of directional change

### 9. Nose Point (node_3) Stability Analysis
- Track 3D position of nose point across all frames
- Compute displacement from mean position (overall and per-axis)
- Measure frame-to-frame displacement and velocity
- Calculate rolling position stability using 1-second windows
- Identify periods of high nose movement

## Outputs

- `normal_vector_stability_analysis.csv`: Frame-by-frame results with all metrics
- `normal_vector_stability_sec_blocks.csv`: Per-second block analysis with angle shifts
- `distribution_statistics.csv`: Summary statistics for all metric distributions
- `worst_case_frames.csv`: Identified outlier frames and worst-case events
- `nose_point_stability.csv`: Frame-by-frame nose point position and stability metrics
- Summary statistics printed to console


In [ ]:
# === PHASE 3 • NORMAL VECTOR STABILITY ANALYSIS ============================
# Analyzes the stability of normal vectors computed from a reference plane
# defined by 3 landmark nodes.
#
# INPUT:
#   csv_path: Path to CSV file with 3D node data (columns: frame, node, x, y, z, time_s)
#   fps: Frames per second (default 120.0)
#   landmark_nodes: List of 3 node names to use for plane computation
#
# OUTPUT:
#   normal_vector_stability_analysis.csv: Frame-by-frame results
#   normal_vector_stability_sec_blocks.csv: Per-second block analysis
# ============================================================================

# ===================== USER CONFIGURATION =====================
csv_path = r"/Users/howardwang/Desktop/Ruten/Evaluation-Metrics_Vishal-main/face/results/n20/triangulated/session_2025-05-28_14-12-04_124591_v1/all_nodes_3d_long_v1.csv"  # Path to 3D node data CSV
fps = 120.0  # Frames per second
landmark_nodes = ['node_1', 'node_2', 'node_3']  # 3 nodes to define reference plane

# Output directory
out_root = r"/Users/howardwang/Desktop/Ruten/Evaluation-Metrics_Vishal-main/face/results/n20/phase3_step1"  # Output directory for results

# Analysis parameters
stability_threshold_deg = 5.0  # Threshold for stable/changing classification (degrees)
lowpass_cutoff_hz = 5.0  # Low-pass filter cutoff frequency (Hz)
# ==============================================================

import pandas as pd
import numpy as np
from pathlib import Path
from scipy import signal
import os

os.makedirs(out_root, exist_ok=True)


In [ ]:
# ===================== HELPER FUNCTIONS =====================

def load_and_structure_data(csv_path):
    """Load CSV data and convert to structured array format"""
    print("Loading data...")
    df = pd.read_csv(csv_path)
    
    # Get frames and nodes
    frames = df['frame'].unique()
    nodes = df['node'].unique()
    print(f"Frames: {len(frames)}, Nodes: {len(nodes)}")
    print(f"Nodes: {nodes}")
    
    # Prepare data array: frames x nodes x coordinates
    n_frames = len(frames)
    n_nodes = len(nodes)
    X3 = np.full((n_frames, n_nodes, 3), np.nan)
    
    # Fill the array
    for i, node in enumerate(nodes):
        node_data = df[df['node'] == node]
        for _, row in node_data.iterrows():
            frame_idx = int(row['frame'])
            if frame_idx < n_frames:
                X3[frame_idx, i, 0] = row['x'] if pd.notna(row['x']) else np.nan
                X3[frame_idx, i, 1] = row['y'] if pd.notna(row['y']) else np.nan
                X3[frame_idx, i, 2] = row['z'] if pd.notna(row['z']) else np.nan
    
    # Get time array
    times = df[df['node'] == nodes[0]]['time_s'].values
    
    return X3, frames, nodes, times


def find_node_indices(nodes, target_nodes):
    """Find indices of target nodes in the nodes array"""
    indices = []
    for target in target_nodes:
        for i, n in enumerate(nodes):
            if n == target:
                indices.append(i)
                break
    return indices


def compute_plane_normal(X3, frame, landmark_indices):
    """
    Compute plane normal vector from 3 landmark points
    
    Args:
        X3: Data array (frames x nodes x coordinates)
        frame: Current frame index
        landmark_indices: Indices of 3 landmark nodes
    
    Returns:
        normal: Normalized plane normal vector (3D) or None if invalid
    """
    # Extract landmark positions
    p1 = X3[frame, landmark_indices[0], :]
    p2 = X3[frame, landmark_indices[1], :]
    p3 = X3[frame, landmark_indices[2], :]
    
    # Check if all points are valid
    if not (np.all(np.isfinite(p1)) and np.all(np.isfinite(p2)) and np.all(np.isfinite(p3))):
        return None
    
    # Calculate plane normal via cross product
    v1 = p2 - p1
    v2 = p3 - p1
    normal = np.cross(v1, v2)
    norm_mag = np.linalg.norm(normal)
    
    if norm_mag < 1e-10:
        return None
    
    normal = normal / norm_mag  # Normalize
    
    return normal


def compute_normal_angle(n1, n2):
    """
    Compute angle between two normal vectors in degrees
    
    Args:
        n1, n2: Normalized normal vectors (3D arrays)
    
    Returns:
        angle: Angle in degrees (0-180)
    """
    if n1 is None or n2 is None:
        return np.nan
    
    if not (np.all(np.isfinite(n1)) and np.all(np.isfinite(n2))):
        return np.nan
    
    # Dot product (clamped to handle numerical errors)
    dot_product = np.clip(np.dot(n1, n2), -1.0, 1.0)
    angle_rad = np.arccos(dot_product)
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg


def rolling_statistics(data, window_size_frames, func=np.mean, fill_value=np.nan):
    """
    Compute rolling statistics over a window
    
    Args:
        data: 1D or 2D array
        window_size_frames: Window size in frames
        func: Function to apply (np.mean, np.std, etc.)
        fill_value: Value to use for insufficient data
    
    Returns:
        Rolling statistics array
    """
    if len(data) == 0:
        return np.array([])
    
    # Handle 2D arrays (for vector data)
    if data.ndim == 2:
        n_frames, n_dims = data.shape
        result = np.full((n_frames, n_dims), fill_value)
        for i in range(n_frames):
            start = max(0, i - window_size_frames // 2)
            end = min(n_frames, i + window_size_frames // 2 + 1)
            window_data = data[start:end, :]
            valid_mask = np.all(np.isfinite(window_data), axis=1)
            if np.sum(valid_mask) > 0:
                result[i, :] = func(window_data[valid_mask], axis=0)
    else:
        # Handle 1D arrays
        n_frames = len(data)
        result = np.full(n_frames, fill_value)
        for i in range(n_frames):
            start = max(0, i - window_size_frames // 2)
            end = min(n_frames, i + window_size_frames // 2 + 1)
            window_data = data[start:end]
            valid_mask = np.isfinite(window_data)
            if np.sum(valid_mask) > 0:
                result[i] = func(window_data[valid_mask])
    
    return result


def apply_lowpass_filter(data, fps, cutoff_hz=5.0, order=4):
    """
    Apply Butterworth low-pass filter to reduce high-frequency noise
    
    Args:
        data: 2D array (n_frames x n_dims)
        fps: Frames per second
        cutoff_hz: Cutoff frequency in Hz
        order: Filter order
    
    Returns:
        Filtered data
    """
    if len(data) == 0:
        return data
    
    nyquist = fps / 2.0
    if cutoff_hz >= nyquist:
        cutoff_hz = nyquist * 0.95  # Ensure below Nyquist
    
    b, a = signal.butter(order, cutoff_hz / nyquist, btype='low')
    
    # Filter each dimension separately
    filtered = np.full_like(data, np.nan)
    for dim in range(data.shape[1]):
        valid_mask = np.isfinite(data[:, dim])
        if np.sum(valid_mask) > order * 2:  # Need enough points for filter
            filtered_data = signal.filtfilt(b, a, data[valid_mask, dim])
            filtered[valid_mask, dim] = filtered_data
    
    return filtered


def compute_stability_metrics(normals):
    """
    Compute overall stability metrics for normal vectors
    
    Args:
        normals: Array of normal vectors (n_frames x 3)
    
    Returns:
        Dictionary with stability metrics
    """
    # Filter out invalid normals
    valid_mask = np.all(np.isfinite(normals), axis=1)
    valid_normals = normals[valid_mask]
    
    if len(valid_normals) == 0:
        return {
            'mean_normal': np.array([np.nan, np.nan, np.nan]),
            'std_components': np.array([np.nan, np.nan, np.nan]),
            'mean_angular_dispersion': np.nan,
            'coefficient_of_variation': np.nan
        }
    
    # Mean normal vector
    mean_normal = np.mean(valid_normals, axis=0)
    mean_normal = mean_normal / np.linalg.norm(mean_normal)  # Normalize
    
    # Standard deviation of components
    std_components = np.std(valid_normals, axis=0)
    
    # Angular dispersion: std dev of angles from mean normal
    angles_from_mean = []
    for n in valid_normals:
        angle = compute_normal_angle(n, mean_normal)
        if not np.isnan(angle):
            angles_from_mean.append(angle)
    
    mean_angular_dispersion = np.std(angles_from_mean) if len(angles_from_mean) > 0 else np.nan
    
    # Coefficient of variation for normal magnitude (should be ~0 for unit vectors)
    magnitudes = np.linalg.norm(valid_normals, axis=1)
    mean_magnitude = np.mean(magnitudes)
    std_magnitude = np.std(magnitudes)
    coefficient_of_variation = (std_magnitude / mean_magnitude) if mean_magnitude > 0 else np.nan
    
    return {
        'mean_normal': mean_normal,
        'std_components': std_components,
        'mean_angular_dispersion': mean_angular_dispersion,
        'coefficient_of_variation': coefficient_of_variation
    }


def classify_state(normals_stability_std, threshold=5.0):
    """
    Classify each frame as stable or changing based on rolling standard deviation
    
    Args:
        normals_stability_std: Rolling standard deviation values
        threshold: Threshold in degrees for classification
    
    Returns:
        Array of state labels ('stable' or 'changing')
    """
    states = np.full(len(normals_stability_std), 'changing', dtype=object)
    valid_mask = np.isfinite(normals_stability_std)
    states[valid_mask & (normals_stability_std < threshold)] = 'stable'
    
    return states
# ==============================================================


In [ ]:
# ===================== MAIN ANALYSIS =====================

print("="*60)
print("Normal Vector Stability Analysis")
print("="*60)

dt = 1.0 / fps

# Window sizes in frames
window_0_5s = int(0.5 * fps)  # 60 frames
window_1s = int(1.0 * fps)    # 120 frames
window_5s = int(5.0 * fps)    # 600 frames
window_10s = int(10.0 * fps)  # 1200 frames

# Load data
X3, frames, nodes, times = load_and_structure_data(csv_path)

# Find landmark indices
landmark_indices = find_node_indices(nodes, landmark_nodes)
if len(landmark_indices) != 3:
    print(f"Error: Could not find all landmarks. Expected {landmark_nodes}")
    print(f"Found indices: {landmark_indices}")
    raise RuntimeError(f"Could not find all landmarks: {landmark_nodes}")

print(f"\nLandmark nodes: {landmark_nodes}")
print(f"Landmark indices: {landmark_indices}")

n_frames = len(frames)

# Compute normal vectors for all frames
print("\nComputing normal vectors...")
normals = np.full((n_frames, 3), np.nan)

for i in range(n_frames):
    normal = compute_plane_normal(X3, i, landmark_indices)
    if normal is not None:
        normals[i, :] = normal

valid_normal_count = np.sum(np.all(np.isfinite(normals), axis=1))
print(f"Computed {valid_normal_count} valid normal vectors out of {n_frames} frames")

# Compute angle changes
print("\nComputing angle changes...")
angle_changes = np.full(n_frames, np.nan)
cumulative_angle_changes = np.full(n_frames, np.nan)
angular_velocities = np.full(n_frames, np.nan)

for i in range(1, n_frames):
    if np.all(np.isfinite(normals[i, :])) and np.all(np.isfinite(normals[i-1, :])):
        angle = compute_normal_angle(normals[i-1, :], normals[i, :])
        angle_changes[i] = angle
        
        # Angular velocity (degrees per second)
        if not np.isnan(angle):
            angular_velocities[i] = angle / dt

# Cumulative angle change from first frame
cumulative_angle = 0.0
first_valid_idx = None
for i in range(n_frames):
    if np.all(np.isfinite(normals[i, :])):
        if first_valid_idx is None:
            first_valid_idx = i
            cumulative_angle_changes[i] = 0.0
        elif first_valid_idx is not None:
            angle = compute_normal_angle(normals[first_valid_idx, :], normals[i, :])
            if not np.isnan(angle):
                cumulative_angle = angle
                cumulative_angle_changes[i] = cumulative_angle

# Rolling statistics
print("\nComputing rolling statistics...")

# Rolling mean normal vectors (0.5s, 1s, 5s windows)
rolling_mean_0_5s = rolling_statistics(normals, window_0_5s, func=np.mean)
rolling_mean_1s = rolling_statistics(normals, window_1s, func=np.mean)
rolling_mean_5s = rolling_statistics(normals, window_5s, func=np.mean)

# Normalize rolling means
for i in range(n_frames):
    if np.all(np.isfinite(rolling_mean_0_5s[i, :])):
        rolling_mean_0_5s[i, :] = rolling_mean_0_5s[i, :] / np.linalg.norm(rolling_mean_0_5s[i, :])
    if np.all(np.isfinite(rolling_mean_1s[i, :])):
        rolling_mean_1s[i, :] = rolling_mean_1s[i, :] / np.linalg.norm(rolling_mean_1s[i, :])
    if np.all(np.isfinite(rolling_mean_5s[i, :])):
        rolling_mean_5s[i, :] = rolling_mean_5s[i, :] / np.linalg.norm(rolling_mean_5s[i, :])

# Rolling standard deviation of angles (using 1s window)
reference_normal = rolling_mean_1s
normals_stability_std = np.full(n_frames, np.nan)
normals_stability_mean_angle = np.full(n_frames, np.nan)

for i in range(n_frames):
    if np.all(np.isfinite(normals[i, :])) and np.all(np.isfinite(reference_normal[i, :])):
        # Angle from current normal to rolling mean
        angle = compute_normal_angle(normals[i, :], reference_normal[i, :])
        normals_stability_mean_angle[i] = angle
        
        # Rolling std of angles in window
        start = max(0, i - window_1s // 2)
        end = min(n_frames, i + window_1s // 2 + 1)
        window_angles = normals_stability_mean_angle[start:end]
        valid_angles = window_angles[np.isfinite(window_angles)]
        if len(valid_angles) > 1:
            normals_stability_std[i] = np.std(valid_angles)

# Low-pass filtered normals
print("Applying low-pass filter...")
filtered_normals = apply_lowpass_filter(normals, fps, cutoff_hz=lowpass_cutoff_hz)
# Normalize filtered normals
for i in range(n_frames):
    if np.all(np.isfinite(filtered_normals[i, :])):
        filtered_normals[i, :] = filtered_normals[i, :] / np.linalg.norm(filtered_normals[i, :])

# State classification
print("Classifying states...")
state_labels = classify_state(normals_stability_std, threshold=stability_threshold_deg)

# Overall stability metrics
print("\nComputing overall stability metrics...")
stability_metrics = compute_stability_metrics(normals)

# Create output DataFrame
print("\nCreating output DataFrame...")
results = []

for i in range(n_frames):
    result_row = {
        'frame': frames[i],
        'time_s': times[i],
        'normal_x': normals[i, 0] if np.isfinite(normals[i, 0]) else np.nan,
        'normal_y': normals[i, 1] if np.isfinite(normals[i, 1]) else np.nan,
        'normal_z': normals[i, 2] if np.isfinite(normals[i, 2]) else np.nan,
        'angle_change_deg': angle_changes[i] if np.isfinite(angle_changes[i]) else np.nan,
        'cumulative_angle_change_deg': cumulative_angle_changes[i] if np.isfinite(cumulative_angle_changes[i]) else np.nan,
        'angular_velocity_deg_per_s': angular_velocities[i] if np.isfinite(angular_velocities[i]) else np.nan,
        'normal_stability_std': normals_stability_std[i] if np.isfinite(normals_stability_std[i]) else np.nan,
        'normal_stability_mean_angle': normals_stability_mean_angle[i] if np.isfinite(normals_stability_mean_angle[i]) else np.nan,
        'state_label': state_labels[i],
        'filtered_normal_x': filtered_normals[i, 0] if np.isfinite(filtered_normals[i, 0]) else np.nan,
        'filtered_normal_y': filtered_normals[i, 1] if np.isfinite(filtered_normals[i, 1]) else np.nan,
        'filtered_normal_z': filtered_normals[i, 2] if np.isfinite(filtered_normals[i, 2]) else np.nan,
    }
    results.append(result_row)

results_df = pd.DataFrame(results)

# Per-second block analysis: mean normal per second and angle shift
print("\nComputing per-second block angle shifts...")
normals_df = pd.DataFrame({
    'time_s': times,
    'normal_x': normals[:, 0],
    'normal_y': normals[:, 1],
    'normal_z': normals[:, 2],
})
normals_df['second'] = np.floor(normals_df['time_s']).astype(int)

sec_rows = []
for sec, grp in normals_df.groupby('second'):
    vals = grp[['normal_x', 'normal_y', 'normal_z']].values
    valid = np.all(np.isfinite(vals), axis=1)
    if valid.sum() == 0:
        continue
    mean_vec = np.mean(vals[valid], axis=0)
    if np.all(np.isfinite(mean_vec)) and np.linalg.norm(mean_vec) > 0:
        mean_vec = mean_vec / np.linalg.norm(mean_vec)
        sec_rows.append({
            'second': int(sec),
            'time_start_s': float(sec),
            'time_end_s': float(sec + 1),
            'mean_normal_x': mean_vec[0],
            'mean_normal_y': mean_vec[1],
            'mean_normal_z': mean_vec[2],
            'num_valid_frames': int(valid.sum()),
            'num_frames_in_sec': int(len(valid))
        })

sec_df = pd.DataFrame(sec_rows).sort_values('second').reset_index(drop=True)

# Enforce continuity (flip sign if necessary) to avoid 180-deg jumps
def angle_deg(n1, n2):
    dp = np.clip(np.dot(n1, n2), -1.0, 1.0)
    return float(np.degrees(np.arccos(dp)))

prev = None
for i in range(len(sec_df)):
    curr = sec_df.loc[i, ['mean_normal_x','mean_normal_y','mean_normal_z']].values.astype(float)
    if prev is not None and np.all(np.isfinite(prev)) and np.all(np.isfinite(curr)):
        if np.dot(prev, curr) < 0:
            curr = -curr
            sec_df.loc[i, ['mean_normal_x','mean_normal_y','mean_normal_z']] = curr
    prev = curr

# Compute sec-to-sec angle shifts
angles = [np.nan]
for i in range(1, len(sec_df)):
    n1 = sec_df.loc[i-1, ['mean_normal_x','mean_normal_y','mean_normal_z']].values.astype(float)
    n2 = sec_df.loc[i,   ['mean_normal_x','mean_normal_y','mean_normal_z']].values.astype(float)
    angles.append(angle_deg(n1, n2))
sec_df['angle_to_prev_sec_deg'] = angles

# Save per-second analysis
sec_out = Path(out_root) / 'normal_vector_stability_sec_blocks.csv'
sec_out.parent.mkdir(exist_ok=True, parents=True)
sec_df.to_csv(sec_out, index=False)
print(f"Per-second block analysis saved to: {sec_out}")

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

valid_mask = np.all(np.isfinite(normals), axis=1)
valid_count = np.sum(valid_mask)

print(f"\nOverall Stability Metrics:")
print(f"  Mean normal vector: [{stability_metrics['mean_normal'][0]:.6f}, "
      f"{stability_metrics['mean_normal'][1]:.6f}, {stability_metrics['mean_normal'][2]:.6f}]")
print(f"  Std dev of components: [{stability_metrics['std_components'][0]:.6f}, "
      f"{stability_metrics['std_components'][1]:.6f}, {stability_metrics['std_components'][2]:.6f}]")
print(f"  Mean angular dispersion: {stability_metrics['mean_angular_dispersion']:.4f} degrees")
print(f"  Coefficient of variation: {stability_metrics['coefficient_of_variation']:.6f}")

print(f"\nAngle Change Statistics:")
valid_angles = angle_changes[np.isfinite(angle_changes)]
if len(valid_angles) > 0:
    print(f"  Mean angle change: {np.mean(valid_angles):.4f} degrees")
    print(f"  Std dev angle change: {np.std(valid_angles):.4f} degrees")
    print(f"  Max angle change: {np.max(valid_angles):.4f} degrees")
    print(f"  Min angle change: {np.min(valid_angles):.4f} degrees")

print(f"\nAngular Velocity Statistics:")
valid_velocities = angular_velocities[np.isfinite(angular_velocities)]
if len(valid_velocities) > 0:
    print(f"  Mean angular velocity: {np.mean(valid_velocities):.4f} deg/s")
    print(f"  Max angular velocity: {np.max(valid_velocities):.4f} deg/s")

# Per-second summary
if len(sec_df) > 1:
    valid_sec_angles = sec_df['angle_to_prev_sec_deg'].dropna()
    if len(valid_sec_angles) > 0:
        print(f"\nPer-second angle shift (sec-to-sec):")
        print(f"  Mean: {valid_sec_angles.mean():.4f} deg/sec")
        print(f"  Median: {valid_sec_angles.median():.4f} deg/sec")
        print(f"  Max: {valid_sec_angles.max():.4f} deg/sec")

print(f"\nState Classification:")
stable_count = np.sum(state_labels == 'stable')
changing_count = np.sum(state_labels == 'changing')
stable_percent = 100 * stable_count / n_frames if n_frames > 0 else 0
changing_percent = 100 * changing_count / n_frames if n_frames > 0 else 0
print(f"  Stable frames: {stable_percent:.2f}% ({stable_count} frames)")
print(f"  Changing frames: {changing_percent:.2f}% ({changing_count} frames)")

# Time periods with highest/lowest stability
valid_stability = normals_stability_std[np.isfinite(normals_stability_std)]
if len(valid_stability) > 0:
    min_stability_idx = np.argmin(valid_stability)
    max_stability_idx = np.argmax(valid_stability)
    min_stability_time = times[np.where(np.isfinite(normals_stability_std))[0][min_stability_idx]]
    max_stability_time = times[np.where(np.isfinite(normals_stability_std))[0][max_stability_idx]]
    
    print(f"\nStability Periods:")
    print(f"  Most stable time: {min_stability_time:.3f} s (std: {np.min(valid_stability):.4f} deg)")
    print(f"  Least stable time: {max_stability_time:.3f} s (std: {np.max(valid_stability):.4f} deg)")

print("="*60)

# Save results
output_path = Path(out_root) / 'normal_vector_stability_analysis.csv'
output_path.parent.mkdir(exist_ok=True, parents=True)
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to: {output_path}")
print("[DONE]")


In [ ]:
# ===================== ACROSS-FRAME DISTRIBUTION ANALYSIS =====================
# Computes and displays distribution statistics for angle changes, normal vector
# components, and angular velocity across all frames.
# ==============================================================================

from scipy import stats

print("="*60)
print("ACROSS-FRAME DISTRIBUTION ANALYSIS")
print("="*60)

# Prepare valid data arrays
valid_angle_changes = angle_changes[np.isfinite(angle_changes)]
valid_angular_velocities = angular_velocities[np.isfinite(angular_velocities)]
valid_normal_x = normals[:, 0][np.isfinite(normals[:, 0])]
valid_normal_y = normals[:, 1][np.isfinite(normals[:, 1])]
valid_normal_z = normals[:, 2][np.isfinite(normals[:, 2])]

distribution_stats = []

# --- 1. Angle Change Distribution ---
print("\n--- Angle Change Distribution ---")
if len(valid_angle_changes) > 0:
    angle_mean = np.mean(valid_angle_changes)
    angle_median = np.median(valid_angle_changes)
    angle_std = np.std(valid_angle_changes)
    angle_min = np.min(valid_angle_changes)
    angle_max = np.max(valid_angle_changes)
    angle_p25 = np.percentile(valid_angle_changes, 25)
    angle_p75 = np.percentile(valid_angle_changes, 75)
    angle_p95 = np.percentile(valid_angle_changes, 95)
    angle_p99 = np.percentile(valid_angle_changes, 99)
    angle_skewness = stats.skew(valid_angle_changes)
    angle_kurtosis = stats.kurtosis(valid_angle_changes)
    
    print(f"  Count: {len(valid_angle_changes)}")
    print(f"  Mean: {angle_mean:.4f} deg")
    print(f"  Median: {angle_median:.4f} deg")
    print(f"  Std Dev: {angle_std:.4f} deg")
    print(f"  Min: {angle_min:.4f} deg")
    print(f"  Max: {angle_max:.4f} deg")
    print(f"  25th percentile: {angle_p25:.4f} deg")
    print(f"  75th percentile: {angle_p75:.4f} deg")
    print(f"  95th percentile: {angle_p95:.4f} deg")
    print(f"  99th percentile: {angle_p99:.4f} deg")
    print(f"  Skewness: {angle_skewness:.4f}")
    print(f"  Kurtosis: {angle_kurtosis:.4f}")
    
    distribution_stats.append({
        'metric': 'angle_change_deg',
        'count': len(valid_angle_changes),
        'mean': angle_mean, 'median': angle_median, 'std': angle_std,
        'min': angle_min, 'max': angle_max,
        'p25': angle_p25, 'p75': angle_p75, 'p95': angle_p95, 'p99': angle_p99,
        'skewness': angle_skewness, 'kurtosis': angle_kurtosis
    })

# --- 2. Angular Velocity Distribution ---
print("\n--- Angular Velocity Distribution ---")
if len(valid_angular_velocities) > 0:
    vel_mean = np.mean(valid_angular_velocities)
    vel_median = np.median(valid_angular_velocities)
    vel_std = np.std(valid_angular_velocities)
    vel_min = np.min(valid_angular_velocities)
    vel_max = np.max(valid_angular_velocities)
    vel_p25 = np.percentile(valid_angular_velocities, 25)
    vel_p75 = np.percentile(valid_angular_velocities, 75)
    vel_p95 = np.percentile(valid_angular_velocities, 95)
    vel_p99 = np.percentile(valid_angular_velocities, 99)
    vel_skewness = stats.skew(valid_angular_velocities)
    vel_kurtosis = stats.kurtosis(valid_angular_velocities)
    
    print(f"  Count: {len(valid_angular_velocities)}")
    print(f"  Mean: {vel_mean:.4f} deg/s")
    print(f"  Median: {vel_median:.4f} deg/s")
    print(f"  Std Dev: {vel_std:.4f} deg/s")
    print(f"  Min: {vel_min:.4f} deg/s")
    print(f"  Max: {vel_max:.4f} deg/s")
    print(f"  25th percentile: {vel_p25:.4f} deg/s")
    print(f"  75th percentile: {vel_p75:.4f} deg/s")
    print(f"  95th percentile: {vel_p95:.4f} deg/s")
    print(f"  99th percentile: {vel_p99:.4f} deg/s")
    print(f"  Skewness: {vel_skewness:.4f}")
    print(f"  Kurtosis: {vel_kurtosis:.4f}")
    
    distribution_stats.append({
        'metric': 'angular_velocity_deg_per_s',
        'count': len(valid_angular_velocities),
        'mean': vel_mean, 'median': vel_median, 'std': vel_std,
        'min': vel_min, 'max': vel_max,
        'p25': vel_p25, 'p75': vel_p75, 'p95': vel_p95, 'p99': vel_p99,
        'skewness': vel_skewness, 'kurtosis': vel_kurtosis
    })

# --- 3. Normal Vector Component Distributions ---
print("\n--- Normal Vector Component Distributions ---")
for comp_name, comp_data in [('normal_x', valid_normal_x), 
                              ('normal_y', valid_normal_y), 
                              ('normal_z', valid_normal_z)]:
    if len(comp_data) > 0:
        comp_mean = np.mean(comp_data)
        comp_median = np.median(comp_data)
        comp_std = np.std(comp_data)
        comp_min = np.min(comp_data)
        comp_max = np.max(comp_data)
        comp_p25 = np.percentile(comp_data, 25)
        comp_p75 = np.percentile(comp_data, 75)
        comp_p95 = np.percentile(comp_data, 95)
        comp_p99 = np.percentile(comp_data, 99)
        comp_skewness = stats.skew(comp_data)
        comp_kurtosis = stats.kurtosis(comp_data)
        
        print(f"\n  {comp_name}:")
        print(f"    Count: {len(comp_data)}")
        print(f"    Mean: {comp_mean:.6f}")
        print(f"    Median: {comp_median:.6f}")
        print(f"    Std Dev: {comp_std:.6f}")
        print(f"    Min: {comp_min:.6f}, Max: {comp_max:.6f}")
        print(f"    Range (p25-p75): [{comp_p25:.6f}, {comp_p75:.6f}]")
        print(f"    Skewness: {comp_skewness:.4f}, Kurtosis: {comp_kurtosis:.4f}")
        
        distribution_stats.append({
            'metric': comp_name,
            'count': len(comp_data),
            'mean': comp_mean, 'median': comp_median, 'std': comp_std,
            'min': comp_min, 'max': comp_max,
            'p25': comp_p25, 'p75': comp_p75, 'p95': comp_p95, 'p99': comp_p99,
            'skewness': comp_skewness, 'kurtosis': comp_kurtosis
        })

# Save distribution statistics
dist_stats_df = pd.DataFrame(distribution_stats)
dist_out = Path(out_root) / 'distribution_statistics.csv'
dist_stats_df.to_csv(dist_out, index=False)
print(f"\nDistribution statistics saved to: {dist_out}")
print("="*60)


In [ ]:
# ===================== WORST-CASE ANALYSIS =====================
# Identifies worst-case scenarios: maximum deviations, outlier frames,
# and periods of high instability.
# ================================================================

print("="*60)
print("WORST-CASE ANALYSIS")
print("="*60)

# Configuration for outlier detection
outlier_std_threshold = 3.0  # Number of standard deviations for outlier detection

worst_case_frames = []

# --- 1. Maximum Angle Deviation from Mean Normal ---
print("\n--- Maximum Angle Deviation from Mean Normal ---")
mean_normal = stability_metrics['mean_normal']
if np.all(np.isfinite(mean_normal)):
    angles_from_mean = np.full(n_frames, np.nan)
    for i in range(n_frames):
        if np.all(np.isfinite(normals[i, :])):
            angles_from_mean[i] = compute_normal_angle(normals[i, :], mean_normal)
    
    valid_angles_from_mean = angles_from_mean[np.isfinite(angles_from_mean)]
    if len(valid_angles_from_mean) > 0:
        max_deviation_idx = np.nanargmax(angles_from_mean)
        max_deviation = angles_from_mean[max_deviation_idx]
        max_deviation_time = times[max_deviation_idx]
        
        print(f"  Maximum deviation: {max_deviation:.4f} degrees")
        print(f"  Frame: {frames[max_deviation_idx]}")
        print(f"  Time: {max_deviation_time:.4f} s")
        
        worst_case_frames.append({
            'frame': frames[max_deviation_idx],
            'time_s': max_deviation_time,
            'type': 'max_deviation_from_mean',
            'value_deg': max_deviation,
            'description': 'Maximum angle deviation from mean normal vector'
        })

# --- 2. Outlier Frame Detection (angle changes > 3 std) ---
print(f"\n--- Outlier Frame Detection (>{outlier_std_threshold} std) ---")
if len(valid_angle_changes) > 0:
    angle_mean = np.mean(valid_angle_changes)
    angle_std = np.std(valid_angle_changes)
    outlier_threshold = angle_mean + outlier_std_threshold * angle_std
    
    print(f"  Mean angle change: {angle_mean:.4f} deg")
    print(f"  Std dev: {angle_std:.4f} deg")
    print(f"  Outlier threshold (>{outlier_std_threshold} std): {outlier_threshold:.4f} deg")
    
    outlier_mask = angle_changes > outlier_threshold
    outlier_indices = np.where(outlier_mask)[0]
    
    print(f"  Number of outlier frames: {len(outlier_indices)}")
    
    if len(outlier_indices) > 0:
        print(f"\n  Top 10 outlier frames:")
        # Sort by angle change descending
        sorted_outliers = sorted(outlier_indices, key=lambda x: angle_changes[x], reverse=True)
        for idx in sorted_outliers[:10]:
            print(f"    Frame {frames[idx]}: {angle_changes[idx]:.4f} deg at t={times[idx]:.4f}s")
            worst_case_frames.append({
                'frame': frames[idx],
                'time_s': times[idx],
                'type': 'outlier_angle_change',
                'value_deg': angle_changes[idx],
                'description': f'Angle change exceeds {outlier_std_threshold} std ({outlier_threshold:.2f} deg)'
            })

# --- 3. Unstable Time Periods ---
print("\n--- Unstable Time Periods ---")
# Find contiguous periods where stability std exceeds threshold
unstable_mask = normals_stability_std >= stability_threshold_deg
unstable_indices = np.where(unstable_mask)[0]

if len(unstable_indices) > 0:
    # Find contiguous regions
    unstable_periods = []
    period_start = unstable_indices[0]
    period_end = unstable_indices[0]
    
    for i in range(1, len(unstable_indices)):
        if unstable_indices[i] == unstable_indices[i-1] + 1:
            period_end = unstable_indices[i]
        else:
            # Save previous period
            unstable_periods.append((period_start, period_end))
            period_start = unstable_indices[i]
            period_end = unstable_indices[i]
    # Save last period
    unstable_periods.append((period_start, period_end))
    
    print(f"  Number of unstable periods: {len(unstable_periods)}")
    
    # Sort by duration (longest first)
    unstable_periods_sorted = sorted(unstable_periods, key=lambda x: x[1] - x[0], reverse=True)
    
    print(f"\n  Top 10 longest unstable periods:")
    for start_idx, end_idx in unstable_periods_sorted[:10]:
        duration_frames = end_idx - start_idx + 1
        duration_s = duration_frames / fps
        start_time = times[start_idx] if start_idx < len(times) else np.nan
        end_time = times[end_idx] if end_idx < len(times) else np.nan
        max_std_in_period = np.nanmax(normals_stability_std[start_idx:end_idx+1])
        
        print(f"    {start_time:.3f}s - {end_time:.3f}s ({duration_s:.3f}s, {duration_frames} frames, max std: {max_std_in_period:.2f} deg)")
        
        worst_case_frames.append({
            'frame': frames[start_idx],
            'time_s': start_time,
            'type': 'unstable_period_start',
            'value_deg': max_std_in_period,
            'description': f'Unstable period: {start_time:.3f}s-{end_time:.3f}s ({duration_s:.3f}s duration)'
        })
else:
    print("  No unstable periods detected.")

# --- 4. Frames with Maximum Angular Velocity ---
print("\n--- Maximum Angular Velocity Frames ---")
if len(valid_angular_velocities) > 0:
    top_velocity_indices = np.argsort(angular_velocities)[::-1][:10]  # Top 10
    print("  Top 10 frames by angular velocity:")
    for idx in top_velocity_indices:
        if np.isfinite(angular_velocities[idx]):
            print(f"    Frame {frames[idx]}: {angular_velocities[idx]:.4f} deg/s at t={times[idx]:.4f}s")
            if angular_velocities[idx] == np.nanmax(angular_velocities):
                worst_case_frames.append({
                    'frame': frames[idx],
                    'time_s': times[idx],
                    'type': 'max_angular_velocity',
                    'value_deg': angular_velocities[idx],
                    'description': 'Maximum angular velocity'
                })

# Save worst-case frames
worst_case_df = pd.DataFrame(worst_case_frames)
worst_case_out = Path(out_root) / 'worst_case_frames.csv'
worst_case_df.to_csv(worst_case_out, index=False)
print(f"\nWorst-case frames saved to: {worst_case_out}")
print(f"Total worst-case events identified: {len(worst_case_frames)}")
print("="*60)


In [ ]:
# ===================== NOSE POINT (NODE_3) STABILITY ANALYSIS =====================
# Analyzes the positional stability of the nose point (node_3) across all frames.
# Computes displacement from mean position, frame-to-frame movement, and rolling
# stability metrics.
# ==================================================================================

print("="*60)
print("NOSE POINT (NODE_3) STABILITY ANALYSIS")
print("="*60)

# Configuration
nose_node_name = 'node_3'  # Nose point node name

# Find nose node index
nose_node_idx = None
for i, n in enumerate(nodes):
    if n == nose_node_name:
        nose_node_idx = i
        break

if nose_node_idx is None:
    print(f"Error: Could not find nose node '{nose_node_name}' in the data.")
    print(f"Available nodes: {nodes}")
else:
    print(f"Nose node: {nose_node_name} (index: {nose_node_idx})")
    
    # Extract nose point positions
    nose_positions = X3[:, nose_node_idx, :]  # (n_frames, 3)
    
    # Count valid frames
    valid_nose_mask = np.all(np.isfinite(nose_positions), axis=1)
    valid_nose_count = np.sum(valid_nose_mask)
    print(f"Valid nose point frames: {valid_nose_count} / {n_frames}")
    
    # --- 1. Mean Position and Displacement from Mean ---
    print("\n--- Mean Position and Displacement ---")
    valid_nose_positions = nose_positions[valid_nose_mask]
    
    if len(valid_nose_positions) > 0:
        mean_nose_position = np.mean(valid_nose_positions, axis=0)
        print(f"  Mean position: [{mean_nose_position[0]:.4f}, {mean_nose_position[1]:.4f}, {mean_nose_position[2]:.4f}]")
        
        # Displacement from mean for each frame
        displacement_from_mean = np.full(n_frames, np.nan)
        displacement_x = np.full(n_frames, np.nan)
        displacement_y = np.full(n_frames, np.nan)
        displacement_z = np.full(n_frames, np.nan)
        
        for i in range(n_frames):
            if np.all(np.isfinite(nose_positions[i, :])):
                diff = nose_positions[i, :] - mean_nose_position
                displacement_from_mean[i] = np.linalg.norm(diff)
                displacement_x[i] = diff[0]
                displacement_y[i] = diff[1]
                displacement_z[i] = diff[2]
        
        valid_displacements = displacement_from_mean[np.isfinite(displacement_from_mean)]
        print(f"  Mean displacement from mean: {np.mean(valid_displacements):.4f}")
        print(f"  Max displacement from mean: {np.max(valid_displacements):.4f}")
        print(f"  Std dev of displacement: {np.std(valid_displacements):.4f}")
        
        # Find frame with maximum displacement
        max_disp_idx = np.nanargmax(displacement_from_mean)
        print(f"  Max displacement frame: {frames[max_disp_idx]} at t={times[max_disp_idx]:.4f}s")
    
    # --- 2. Frame-to-Frame Displacement ---
    print("\n--- Frame-to-Frame Displacement ---")
    frame_to_frame_disp = np.full(n_frames, np.nan)
    
    for i in range(1, n_frames):
        if np.all(np.isfinite(nose_positions[i, :])) and np.all(np.isfinite(nose_positions[i-1, :])):
            diff = nose_positions[i, :] - nose_positions[i-1, :]
            frame_to_frame_disp[i] = np.linalg.norm(diff)
    
    valid_f2f_disp = frame_to_frame_disp[np.isfinite(frame_to_frame_disp)]
    if len(valid_f2f_disp) > 0:
        print(f"  Mean frame-to-frame displacement: {np.mean(valid_f2f_disp):.4f}")
        print(f"  Max frame-to-frame displacement: {np.max(valid_f2f_disp):.4f}")
        print(f"  Std dev: {np.std(valid_f2f_disp):.4f}")
        
        # Velocity (displacement per second)
        nose_velocity = frame_to_frame_disp * fps
        valid_velocities = nose_velocity[np.isfinite(nose_velocity)]
        print(f"  Mean velocity: {np.mean(valid_velocities):.4f} units/s")
        print(f"  Max velocity: {np.max(valid_velocities):.4f} units/s")
    
    # --- 3. Rolling Position Stability (1s window) ---
    print("\n--- Rolling Position Stability (1s window) ---")
    nose_rolling_std = np.full(n_frames, np.nan)
    
    for i in range(n_frames):
        start = max(0, i - window_1s // 2)
        end = min(n_frames, i + window_1s // 2 + 1)
        window_positions = nose_positions[start:end, :]
        valid_window = np.all(np.isfinite(window_positions), axis=1)
        
        if np.sum(valid_window) > 1:
            valid_pos = window_positions[valid_window]
            # Compute std of distances from window mean
            window_mean = np.mean(valid_pos, axis=0)
            distances = np.linalg.norm(valid_pos - window_mean, axis=1)
            nose_rolling_std[i] = np.std(distances)
    
    valid_rolling_std = nose_rolling_std[np.isfinite(nose_rolling_std)]
    if len(valid_rolling_std) > 0:
        print(f"  Mean rolling std: {np.mean(valid_rolling_std):.4f}")
        print(f"  Max rolling std: {np.max(valid_rolling_std):.4f}")
        
        # Find periods of high movement
        high_movement_threshold = np.mean(valid_rolling_std) + 2 * np.std(valid_rolling_std)
        high_movement_mask = nose_rolling_std > high_movement_threshold
        high_movement_count = np.sum(high_movement_mask)
        high_movement_percent = 100 * high_movement_count / n_frames
        print(f"  High movement threshold: {high_movement_threshold:.4f}")
        print(f"  High movement frames: {high_movement_count} ({high_movement_percent:.2f}%)")
    
    # --- 4. Nose Point Stability Summary ---
    print("\n--- Nose Point Stability Summary ---")
    
    # Classify nose stability
    if len(valid_rolling_std) > 0:
        stable_nose_threshold = np.median(valid_rolling_std) + np.std(valid_rolling_std)
        stable_nose_count = np.sum(nose_rolling_std <= stable_nose_threshold)
        stable_nose_percent = 100 * stable_nose_count / n_frames
        print(f"  Stable nose frames (<= {stable_nose_threshold:.4f}): {stable_nose_count} ({stable_nose_percent:.2f}%)")
    
    # --- 5. Create and Save Nose Point Stability DataFrame ---
    nose_results = []
    for i in range(n_frames):
        nose_results.append({
            'frame': frames[i],
            'time_s': times[i],
            'nose_x': nose_positions[i, 0] if np.isfinite(nose_positions[i, 0]) else np.nan,
            'nose_y': nose_positions[i, 1] if np.isfinite(nose_positions[i, 1]) else np.nan,
            'nose_z': nose_positions[i, 2] if np.isfinite(nose_positions[i, 2]) else np.nan,
            'displacement_from_mean': displacement_from_mean[i] if np.isfinite(displacement_from_mean[i]) else np.nan,
            'displacement_x': displacement_x[i] if np.isfinite(displacement_x[i]) else np.nan,
            'displacement_y': displacement_y[i] if np.isfinite(displacement_y[i]) else np.nan,
            'displacement_z': displacement_z[i] if np.isfinite(displacement_z[i]) else np.nan,
            'frame_to_frame_disp': frame_to_frame_disp[i] if np.isfinite(frame_to_frame_disp[i]) else np.nan,
            'velocity_units_per_s': nose_velocity[i] if np.isfinite(nose_velocity[i]) else np.nan,
            'rolling_std_1s': nose_rolling_std[i] if np.isfinite(nose_rolling_std[i]) else np.nan,
        })
    
    nose_df = pd.DataFrame(nose_results)
    nose_out = Path(out_root) / 'nose_point_stability.csv'
    nose_df.to_csv(nose_out, index=False)
    print(f"\nNose point stability analysis saved to: {nose_out}")

print("="*60)
print("[NOSE POINT ANALYSIS COMPLETE]")
